# Import Required Libraries
Import the necessary libraries, including TensorFlow, Keras, and other required packages.

In [1]:
# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

# Import other required packages
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

# Load and Preprocess Dataset
Load the mini_pix2pix dataset from the data folder and preprocess the images for training.

In [ ]:
# Load and Preprocess Dataset

# Define the path to the dataset
data_path = 'data/mini_pix2pix/train'

# Function to load and preprocess images
def load_image(image_path):
    image = Image.open(image_path)
    image = image.resize((256, 256))  # Resize images to 256x256
    image = np.array(image) / 127.5 - 1  # Normalize images to [-1, 1]
    return image

# Load the dataset
input_images = []
target_images = []

for image_name in os.listdir(data_path):
    if 'input' in image_name:
        input_images.append(load_image(os.path.join(data_path, image_name)))
    elif 'target' in image_name:
        target_images.append(load_image(os.path.join(data_path, image_name)))

# Convert lists to numpy arrays
input_images = np.array(input_images)
target_images = np.array(target_images)

# Define the Generator Model
Define the architecture of the generator model using Keras.

In [ ]:
# Define the Generator Model
def build_generator():
    inputs = layers.Input(shape=[256, 256, 3])

    # Encoder
    down1 = layers.Conv2D(64, (4, 4), strides=2, padding='same', use_bias=False)(inputs)
    down1 = layers.BatchNormalization()(down1)
    down1 = layers.LeakyReLU()(down1)

    down2 = layers.Conv2D(128, (4, 4), strides=2, padding='same', use_bias=False)(down1)
    down2 = layers.BatchNormalization()(down2)
    down2 = layers.LeakyReLU()(down2)

    down3 = layers.Conv2D(256, (4, 4), strides=2, padding='same', use_bias=False)(down2)
    down3 = layers.BatchNormalization()(down3)
    down3 = layers.LeakyReLU()(down3)

    down4 = layers.Conv2D(512, (4, 4), strides=2, padding='same', use_bias=False)(down3)
    down4 = layers.BatchNormalization()(down4)
    down4 = layers.LeakyReLU()(down4)

    down5 = layers.Conv2D(512, (4, 4), strides=2, padding='same', use_bias=False)(down4)
    down5 = layers.BatchNormalization()(down5)
    down5 = layers.LeakyReLU()(down5)

    down6 = layers.Conv2D(512, (4, 4), strides=2, padding='same', use_bias=False)(down5)
    down6 = layers.BatchNormalization()(down6)
    down6 = layers.LeakyReLU()(down6)

    down7 = layers.Conv2D(512, (4, 4), strides=2, padding='same', use_bias=False)(down6)
    down7 = layers.BatchNormalization()(down7)
    down7 = layers.LeakyReLU()(down7)

    down8 = layers.Conv2D(512, (4, 4), strides=2, padding='same', use_bias=False)(down7)
    down8 = layers.BatchNormalization()(down8)
    down8 = layers.LeakyReLU()(down8)

    # Decoder
    up1 = layers.Conv2DTranspose(512, (4, 4), strides=2, padding='same', use_bias=False)(down8)
    up1 = layers.BatchNormalization()(up1)
    up1 = layers.Dropout(0.5)(up1)
    up1 = layers.ReLU()(up1)
    up1 = layers.Concatenate()([up1, down7])

    up2 = layers.Conv2DTranspose(512, (4, 4), strides=2, padding='same', use_bias=False)(up1)
    up2 = layers.BatchNormalization()(up2)
    up2 = layers.Dropout(0.5)(up2)
    up2 = layers.ReLU()(up2)
    up2 = layers.Concatenate()([up2, down6])

    up3 = layers.Conv2DTranspose(512, (4, 4), strides=2, padding='same', use_bias=False)(up2)
    up3 = layers.BatchNormalization()(up3)
    up3 = layers.Dropout(0.5)(up3)
    up3 = layers.ReLU()(up3)
    up3 = layers.Concatenate()([up3, down5])

    up4 = layers.Conv2DTranspose(512, (4, 4), strides=2, padding='same', use_bias=False)(up3)
    up4 = layers.BatchNormalization()(up4)
    up4 = layers.ReLU()(up4)
    up4 = layers.Concatenate()([up4, down4])

    up5 = layers.Conv2DTranspose(256, (4, 4), strides=2, padding='same', use_bias=False)(up4)
    up5 = layers.BatchNormalization()(up5)
    up5 = layers.ReLU()(up5)
    up5 = layers.Concatenate()([up5, down3])

    up6 = layers.Conv2DTranspose(128, (4, 4), strides=2, padding='same', use_bias=False)(up5)
    up6 = layers.BatchNormalization()(up6)
    up6 = layers.ReLU()(up6)
    up6 = layers.Concatenate()([up6, down2])

    up7 = layers.Conv2DTranspose(64, (4, 4), strides=2, padding='same', use_bias=False)(up6)
    up7 = layers.BatchNormalization()(up7)
    up7 = layers.ReLU()(up7)
    up7 = layers.Concatenate()([up7, down1])

    up8 = layers.Conv2DTranspose(3, (4, 4), strides=2, padding='same', use_bias=False)(up7)
    outputs = layers.Activation('tanh')(up8)

    return Model(inputs=inputs, outputs=outputs)

# Instantiate the generator model
generator = build_generator()
generator.summary()

# Define the Discriminator Model
Define the architecture of the discriminator model using Keras.

In [ ]:
# Define the Discriminator Model
def build_discriminator():
    input_image = layers.Input(shape=[256, 256, 3], name='input_image')
    target_image = layers.Input(shape=[256, 256, 3], name='target_image')

    # Concatenate the input and the target images
    x = layers.Concatenate()([input_image, target_image])

    down1 = layers.Conv2D(64, (4, 4), strides=2, padding='same', use_bias=False)(x)
    down1 = layers.LeakyReLU()(down1)

    down2 = layers.Conv2D(128, (4, 4), strides=2, padding='same', use_bias=False)(down1)
    down2 = layers.BatchNormalization()(down2)
    down2 = layers.LeakyReLU()(down2)

    down3 = layers.Conv2D(256, (4, 4), strides=2, padding='same', use_bias=False)(down2)
    down3 = layers.BatchNormalization()(down3)
    down3 = layers.LeakyReLU()(down3)

    down4 = layers.Conv2D(512, (4, 4), strides=1, padding='same', use_bias=False)(down3)
    down4 = layers.BatchNormalization()(down4)
    down4 = layers.LeakyReLU()(down4)

    down5 = layers.Conv2D(512, (4, 4), strides=2, padding='same', use_bias=False)(down4)
    down5 = layers.BatchNormalization()(down5)
    down5 = layers.LeakyReLU()(down5)

    down6 = layers.Conv2D(512, (4, 4), strides=1, padding='same', use_bias=False)(down5)
    down6 = layers.BatchNormalization()(down6)
    down6 = layers.LeakyReLU()(down6)

    # Output layer
    outputs = layers.Conv2D(1, (4, 4), strides=1, padding='same')(down6)

    return Model(inputs=[input_image, target_image], outputs=outputs)

# Instantiate the discriminator model
discriminator = build_discriminator()
discriminator.summary()

# Define the GAN Model
Combine the generator and discriminator models to define the GAN model.

In [ ]:
# Define the GAN Model
def define_gan(generator, discriminator):
    # Make the discriminator not trainable when combined with the generator
    discriminator.trainable = False

    # Define the input for the GAN model
    input_image = layers.Input(shape=[256, 256, 3])
    target_image = layers.Input(shape=[256, 256, 3])

    # Generate the fake image using the generator
    generated_image = generator(input_image)

    # Get the discriminator's output for the fake image
    gan_output = discriminator([input_image, generated_image])

    # Define the GAN model
    gan = Model(inputs=[input_image, target_image], outputs=[gan_output, generated_image])

    return gan

# Instantiate the GAN model
gan = define_gan(generator, discriminator)
gan.summary()

# Compile and Train the Models
Compile the models and train the GAN using the preprocessed dataset.

In [ ]:
# Compile and Train the Models

# Compile the discriminator model
discriminator.compile(optimizer=tf.keras.optimizers.Adam(2e-4, beta_1=0.5),
                      loss='binary_crossentropy')

# Compile the GAN model
gan.compile(optimizer=tf.keras.optimizers.Adam(2e-4, beta_1=0.5),
            loss=['binary_crossentropy', 'mae'],
            loss_weights=[1, 100])

# Define the training function
def train_gan(generator, discriminator, gan, input_images, target_images, epochs=100, batch_size=1):
    for epoch in range(epochs):
        for i in range(len(input_images)):
            # Select a random batch of images
            idx = np.random.randint(0, len(input_images), batch_size)
            real_input = input_images[idx]
            real_target = target_images[idx]

            # Generate fake images
            fake_target = generator.predict(real_input)

            # Create labels for real and fake images
            real_labels = np.ones((batch_size, 16, 16, 1))
            fake_labels = np.zeros((batch_size, 16, 16, 1))

            # Train the discriminator
            d_loss_real = discriminator.train_on_batch([real_input, real_target], real_labels)
            d_loss_fake = discriminator.train_on_batch([real_input, fake_target], fake_labels)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train the generator
            g_loss = gan.train_on_batch([real_input, real_target], [real_labels, real_target])

        # Print the progress
        print(f'Epoch: {epoch+1}, D Loss: {d_loss}, G Loss: {g_loss}')

# Train the GAN
train_gan(generator, discriminator, gan, input_images, target_images, epochs=100, batch_size=1)

# Generate and Visualize Translated Images
Use the trained generator model to generate translated images and visualize the results.

In [ ]:
# Generate and Visualize Translated Images

# Function to generate and visualize translated images
def generate_and_visualize(generator, input_images, target_images, num_images=5):
    plt.figure(figsize=(15, 10))
    for i in range(num_images):
        # Select a random image from the dataset
        idx = np.random.randint(0, len(input_images))
        input_image = input_images[idx]
        target_image = target_images[idx]

        # Generate the translated image
        translated_image = generator.predict(np.expand_dims(input_image, axis=0))[0]

        # Plot the input image
        plt.subplot(num_images, 3, i * 3 + 1)
        plt.imshow((input_image + 1) / 2)  # Convert back to [0, 1] range for display
        plt.title('Input Image')
        plt.axis('off')

        # Plot the target image
        plt.subplot(num_images, 3, i * 3 + 2)
        plt.imshow((target_image + 1) / 2)  # Convert back to [0, 1] range for display
        plt.title('Target Image')
        plt.axis('off')

        # Plot the translated image
        plt.subplot(num_images, 3, i * 3 + 3)
        plt.imshow((translated_image + 1) / 2)  # Convert back to [0, 1] range for display
        plt.title('Translated Image')
        plt.axis('off')

    plt.show()

# Generate and visualize translated images
generate_and_visualize(generator, input_images, target_images, num_images=5)